# PYTHON 資料科學期末專題 (二)

[提示]

用Pandas讀入時有時需要注意encoding參數

1.1 引入 Pandas Profiling

1.2 需要注意異常值與缺失值的處理, 注意資料區間, 評估值區間差異過大的問題

1.3 資料的分類, 以期可以分別繪製比對圖形; 

[基本目標]

把 Netflix 的資訊分門別列出來


[進階目標]

畫出Heatmap 與 文字雲(找出最多人看的影片)

# 參考範例

同學可以參照範例執行, 有問題隨時提問

In [1]:
!pip install pandas_profiling

In [2]:
# 載入繪圖相關套件
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
# 載入資料處理相關套件
from pandas_profiling import ProfileReport
import numpy as np 
import pandas as pd 

In [4]:
#載入資料集
df = pd.read_csv("netflix_titles.csv")
#檢視資料的項目
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [5]:
#查看資料集
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


# 導入pandas-profiling

import pandas_profiling

生成的數據報告

data是需要生成報告的數據，DataFrame型別

pandas_profiling.ProfileReport(data)

導出報告

pfr = pandas_profiling.ProfileReport(data)
pfr.to_file('report.html')

In [ ]:
#生成資料集的報告, 可以用來檢核可視化結果是否正確
report= ProfileReport(df, title='Netflix Profile Report', explorative=True)
#report.to_file('Netflix.html')
#report.to_widgets()
#report.to_notebook_iframe()

# 資料清洗

先確認缺失值狀況

In [ ]:
# 功能：計算axis=0上的不同觀察結果, 主要是 Non NA
df.nunique()

In [ ]:
#計算缺失值
df.isna().sum()

### 修正 "rating" 這個欄位上的缺失狀況

In [ ]:
df['rating'].unique()

In [ ]:
df[df['rating'].isnull()]

# 基於 Netflix shows,我們可以嘗試使用 replacements

In [ ]:
rating_replacements = {
    211: 'TV-14',
    2411: 'TV-14',
    3288: 'PG-13',
    4056: 'TV-G',
    4402: 'TV-G',
    4403: 'TV-G',
    4706: 'TV-14',
    5015: 'TV-14',
    5234: 'TV-14',
    6231: 'TV-Y'
}

for id, rate in rating_replacements.items():
    df.iloc[id, 8] = rate
    
df['rating'].isna().sum()

### 為了簡單處理, 先行丟棄director 跟 cast

cast: 太多異常值, 因為太多演員

In [ ]:
df = df.drop(['director', 'cast'], axis=1)
df.columns

### 修正date_added 缺失狀況 

In [ ]:
df[df['date_added'].isna()]

I have search for them in the internet and is dificult to find the `date_added`to Netlix, so i will drop them, as they are only 11 rows

In [ ]:
df = df[df['date_added'].notna()]
df['country'].mode()[0]

### 修正country 類別的狀況

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

### 確認缺失值的處理是否完整

In [ ]:
df.isna().sum()

## 產生新的特徵

In [ ]:
df.head()

### 利用日期相加取得新的年月資料

我們將從date_add中獲取年份和月份，以便我們可以比較最有價值的月份在Show realse與Netflix我們新增日期的部分

In [ ]:
df['year_added'] = df['date_added'].apply(lambda x: x.split(" ")[-1])
df['year_added'].head()

In [ ]:
df['month_added'] = df['date_added'].apply(lambda x: x.split(" ")[0])
df['month_added'].head()

In [ ]:
### 根據評分顯示目標

這可能很有趣，表明目標主要是productions

In [ ]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

In [ ]:
df['target_ages'] = df['rating'].replace(ratings_ages)
df['target_ages'].unique()

### Country 

在某些條目中，國家具有多個值。所以將在第一列中添加一個新列，以便我們檢查witch區域是否有更多作品

In [ ]:
# Lets retrieve just the first country
df['principal_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['principal_country'].head()

## 修正日期型別

In [ ]:
df.dtypes

In [ ]:
# type should be a category
df['type'] = pd.Categorical(df['type'])
# target_ages is another category (4 classes)
df['target_ages'] = pd.Categorical(df['target_ages'], categories=['Kids', 'Older Kids', 'Teens', 'Adults'])
# Year added should be integer so we can compare with `released_year`
df['year_added'] = pd.to_numeric(df['year_added'])

In [ ]:
df.dtypes

# 資料視覺化

In [ ]:
df.head()

In [ ]:
df['genre'] = df['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 
df['genre'].head()

In [ ]:
movie_df = df[df['type'] == 'Movie']
show_df = df[df['type'] == 'TV Show']

show_df.head()

# TV shows 跟 Movies的比例

In [ ]:
df['type'].value_counts().reset_index()

In [ ]:
fig = px.pie(df['type'].value_counts().reset_index(), values='type', names='index')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
#新增rating 的資料子集
#方便做年齡層的分布查詢
def generate_rating_df(df):
    rating_df = df.groupby(['rating', 'target_ages']).agg({'show_id': 'count'}).reset_index()
    rating_df = rating_df[rating_df['show_id'] != 0]
    rating_df.columns = ['rating', 'target_ages', 'counts']
    rating_df = rating_df.sort_values('target_ages')
    return rating_df

In [ ]:
rating_df = generate_rating_df(df)
fig = px.bar(rating_df, x='rating', y='counts', color='target_ages')
fig.show()

In [ ]:
show_rating_df
show_rating_df['target_ages'].values
np.arange(show_rating_df['target_ages'].nunique())

In [ ]:
#針對影集或是歌劇
movie_rating_df = generate_rating_df(movie_df)
show_rating_df = generate_rating_df(show_df)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(
    go.Pie(labels=movie_rating_df['target_ages'], values=movie_rating_df['counts']),
    row=1, col=1
)

fig.add_trace(
    go.Pie( labels=show_rating_df['target_ages'], values=show_rating_df['counts']),
           row=1, col=2
)

fig.update_traces(textposition='inside', hole=0.3, hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Rating distribution by Type of content",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Movies', x=0.18, y=0.5, font_size=12, showarrow=False),
                 dict(text='TV Shows', x=0.83, y=0.5, font_size=12, showarrow=False)])
fig.show()

In [ ]:
#按國別分別顯示影集與歌劇的產出分布,使用圓餅圖
country_df = df['principal_country'].value_counts().reset_index()
country_df = country_df[country_df['principal_country'] /  country_df['principal_country'].sum() > 0.01]

fig = px.pie(country_df, values='principal_country', names='index')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
#按國別分別顯示影集與歌劇的產出分布,使用歷史分布
fig = px.histogram(df, x='principal_country')
fig.update_xaxes(categoryorder='total descending')
fig.show()

## 發佈內容
我們可以看到，在過去的幾年中，內容製作的數量在增加

In [ ]:
print(released_year_df )
print(added_year_df)

In [ ]:
released_year_df = df.loc[df['release_year'] > 2010].groupby(['release_year', 'type']).agg({'show_id': 'count'}).reset_index()
added_year_df = df.loc[df['year_added'] > 2010].groupby(['year_added', 'type']).agg({'show_id': 'count'}).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter( 
    x=released_year_df.loc[released_year_df['type'] == 'Movie']['release_year'], 
    y=released_year_df.loc[released_year_df['type'] == 'Movie']['show_id'],
    mode='lines+markers',
    name='Movie: Released Year',
    marker_color='green',
))
fig.add_trace(go.Scatter( 
    x=released_year_df.loc[released_year_df['type'] == 'TV Show']['release_year'], 
    y=released_year_df.loc[released_year_df['type'] == 'TV Show']['show_id'],
    mode='lines+markers',
    name='TV Show: Released Year',
    marker_color='darkgreen',
))
fig.add_trace(go.Scatter( 
    x=added_year_df.loc[added_year_df['type'] == 'Movie']['year_added'], 
    y=added_year_df.loc[added_year_df['type'] == 'Movie']['show_id'],
    mode='lines+markers',
    name='Movie: Year Added',
    marker_color='orange',
))
fig.add_trace(go.Scatter( 
    x=added_year_df.loc[added_year_df['type'] == 'TV Show']['year_added'], 
    y=added_year_df.loc[added_year_df['type'] == 'TV Show']['show_id'],
    mode='lines+markers',
    name='TV Show: Year Added',
    marker_color='darkorange',
))
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [ ]:

from scipy.stats import norm

sns.distplot(df.loc[df['release_year'] > 2000, 'release_year'], fit=norm, kde=False)

#displot()集合了matplotlib的hist()與核函數估計kdeplot的功能，
#增加了rugplot分佈觀測條顯示與利用scipy庫fit擬合參數分佈的新穎用途。
#fit：控制擬合的參數分佈圖形，能夠直觀地評估它與觀察數據的對應關係(黑色線條為確定的分佈)

## 建立內容類型的數據框

In [ ]:
#導入SKLEARN的前處理套件, 將元素進行二元變換 , 輸入的資料會轉換成一個 一維 classes_
#每組資料會轉換成和 classes_ 相同大小的一維陣列
#如果資料有對應到 classes_  就為 1 否則為 0

from sklearn.preprocessing import MultiLabelBinarizer 

In [ ]:
def calculate_mlb(series):
    mlb = MultiLabelBinarizer()
    mlb_df = pd.DataFrame(mlb.fit_transform(series), columns=mlb.classes_, index=series.index)
    return mlb_df

In [ ]:
#如何在csv文件中將一列拆分為單獨的列
def top_genres(df, title='Top ones'):
    genres_df = calculate_mlb(df['genre'])
    tdata = genres_df.sum().sort_values(ascending=False)
    print(tdata)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=tdata.index,
        y=tdata.values,
    ))
    fig.update_xaxes(categoryorder='total descending')
    fig.update_layout(title=title)
    fig.show()

In [ ]:
top_genres(movie_df, title='Top Movies Genres')

In [ ]:
top_genres(show_df, title='Top TV Shows Genres')

In [ ]:
#如何在csv文件中將一列拆分為單獨的列
genres_df = calculate_mlb(movie_df['genre'])

movie_corr = genres_df.corr()
movie_mask = np.zeros_like(movie_corr, dtype=np.bool)
movie_mask[np.triu_indices_from(movie_mask)] = True
fig, ax = plt.subplots(figsize=(10, 7))
pl = sns.heatmap(movie_corr,annot=True,fmt=".2f", mask=movie_mask,cmap= "coolwarm",center=0, vmax=1, vmin=-1, linewidths=.5,
                  cbar_kws={"shrink": 0.6,"extend":"both","label":"correlation"},annot_kws={'size': 7})
plt.show()

In [ ]:
genres_df = calculate_mlb(show_df['genre'])

show_corr = genres_df.corr()
show_mask = np.zeros_like(show_corr, dtype=np.bool)
show_mask[np.triu_indices_from(show_mask)] = True
fig, ax = plt.subplots(figsize=(10, 7))
pl =sns.heatmap(show_corr,annot=True,fmt=".2f", mask=show_mask,cmap= "coolwarm",center=0, vmax=1, vmin=-1, linewidths=.5,
                  cbar_kws={"shrink": 0.6,"extend":"both","label":"correlation"},annot_kws={'size': 7})
plt.show()

In [ ]:
#載入文字雲套件
!pip install wordcloud

In [ ]:
#使用文字雲, 找出最多被使用在資料集上的描述
#對文本數據中出現頻率較高的“關鍵詞”在視覺上的突出呈現，形成關鍵詞的渲染形成類似雲一樣的彩色圖片，從而一眼就可以領略文本數據的主要表達意思
from wordcloud import WordCloud
#print(movie_df['genre'])
text = str(list(movie_df['genre'])).replace(',', '').replace('[', '').replace("'", '').replace(']', '')
#print(text)
plt.rcParams['figure.figsize'] = (15, 15)
wordcloud = WordCloud(background_color = 'white', width = 1200,  height = 1200, max_words = 121).generate(text)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
##將背景設為蘋果Logo
from wordcloud import WordCloud
from PIL import Image
#print(movie_df['genre'])
text = str(list(movie_df['genre'])).replace(',', '').replace('[', '').replace("'", '').replace(']', '')
#print(text)
plt.rcParams['figure.figsize'] = (15, 15)
mask=np.array(Image.open(".\\Final-Netflix\\apple.jpg"))
wordcloud = WordCloud(background_color = 'white',mask=mask,  max_words = 121).generate(text)
plt.imshow(wordcloud)
plt.axis('off')
#plt.savefig("apple_word_cloud.png",dpi=360)
plt.show()

### 分析TV SHOW系列持續時間

In [ ]:
fig = px.histogram(x=show_df['duration'])
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(
    title="Distribution of shows duration",
    xaxis_title="Duration of the Show",
)
fig.show()